# Deploy Pre-trained Summarization Model to SageMaker

This notebook deploys a pre-trained BART model to SageMaker for text summarization.

In [ ]:
import boto3
from sagemaker.huggingface import HuggingFaceModel
from sagemaker import get_execution_role

# Get execution role
role = get_execution_role()
hub = {
  'HF_MODEL_ID': 'sshleifer/distilbart-cnn-6-6',
  'HF_TASK':     'summarization'
}

# First delete the existing endpoint configuration
sm_client = boto3.client('sagemaker')
endpoint_name = "huggingface-pytorch-inference-2025-05-08-18-46-16-895"
try:
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"Deleted existing endpoint configuration: {endpoint_name}")
except Exception as e:
    print(f"No existing endpoint configuration found or error: {e}")

# Create the model
huggingface_model = HuggingFaceModel(
    transformers_version='4.26',  # or newer
    pytorch_version='1.13',       # or newer
    py_version='py39',
    env=hub,
    role=role
)

# Deploy with the fixed endpoint name
predictor = huggingface_model.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.xlarge",
    endpoint_name=endpoint_name
)

Deleted existing endpoint configuration: huggingface-pytorch-inference-2025-05-08-18-46-16-895


[05/08/25 20:56:35] INFO     Creating model with name:                                              ]8;id=238494;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=30501;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-05-08-20-56-35-732                                 

[05/08/25 20:56:36] INFO     Creating endpoint-config with name                                     ]8;id=774925;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=84894;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#5937\5937]8;;\
                             huggingface-pytorch-inference-2025-05-08-18-46-16-895                                 

                    INFO     Creating endpoint with name                                            ]8;id=383067;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=637339;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4759\4759]8;;\
                             huggingface-pytorch-inference-2025-05-08-18-46-16-895                                 

## Test the Endpoint

Let's test our deployed model with a simple text sample.

In [10]:
import json, boto3

client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName='huggingface-pytorch-inference-2025-05-08-18-46-16-895',
    Body=json.dumps({"inputs": "The Roleplaying Game is endless. You could explore the horrifying and puzzling experiments of Vault-Tec; become mercenaries, protecting or preying on trade caravans and settlements; trying to make it to the big time with enough caps to live in the post-apocalyptic equivalent of luxury. Find yourself up against the illusive and dangerous Institute who create synth replicas while hiding beneath the ruins or meet with the mighty Brotherhood of Steel and their advanced technology. Or perhaps you are just a group of wanderers, searching for your place in the wasteland and trying to rebuild humanity."}),
    ContentType='application/json'
)
result = json.loads(response['Body'].read())
print(result)
# Example output: [{"summary_text": "Your summarized text..."}]


[{'summary_text': ' The Roleplaying Game is endless. You could explore the horrifying and puzzling experiments of Vault-Tec . Find yourself up against the illusive and dangerous Institute who create synth replicas while hiding beneath the ruins . Or perhaps you are just a group of wanderers, searching for your place in the wasteland .'}]
